# Importsand preparations

In [ ]:
import pandas as pd
import torch
import os
import numpy as np
import datasets
import transformers
from GPUtil import showUtilization as gpu_usage
from numba import cuda
import torch.nn.functional as F

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)

from datasets import load_dataset, DatasetDict

In [ ]:
# !watch -n 0.5 nvidia-smi

In [ ]:
print(f'PyTorch version: {torch.__version__}')  # 1.9.1+cu111
print(f'CUDA version: {torch.version.cuda}')  # 11.1
print(f'cuDNN version: {torch.backends.cudnn.version()}')  # 8005
print(f'Current device: {torch.cuda.current_device()}')  # 0
print(f'Is cuda available: {torch.cuda.is_available()}')  # TRUE

In [ ]:
print(f'Transformers version: {transformers.__version__}')
print(f'Datasets version: {datasets.__version__}')

In [ ]:
# Prevent a warning related to the tokenization process in the transformers library. 
os.environ["TOKENIZERS_PARALLELISM"] = "False"
# Makes CUDA operations synchronous
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [ ]:
# Find the GPU with the least memory usage.
!nvidia-smi

In [ ]:
def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    # free unreferenced tensors from the GPU memory.
    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()   

In [ ]:
# Smaller and faster than bert.
model_ckpt = "distilbert-base-uncased"

epochs = 5 #Number of full cycles through the training set.
num_labels = 2 #Number of labels, high, med, low priority.
learning_rate = 5e-5 # Rate the model updates based on the data its trained on.
train_batch_size = 16 # Number of training examples in one iteration.
eval_batch_size = 32 # Number evaluation examples in on iteration.
save_strategy = "no" # Should the model be saved automatically during training.
save_steps = 500 # How often to save the model during training. No effect since no over.
logging_steps = 100
model_dir = "./model" #Where to save model

# Use early stopping to prevent overfitting
#load_best_model_at_end=True
#metric_for_best_model="eval_loss"
#greater_is_better=False

Load dataset from huggingface

In [ ]:
dataset = load_dataset("kristmh/high_vs_non_high")
dataset

In [ ]:
rust = load_dataset("kristmh/rust_testset_with_med_low")
rust

In [ ]:
flutter = load_dataset("kristmh/flutter_testset_with_med_low")
flutter

In [ ]:
mypy = load_dataset("kristmh/mypy_testset_with_med_low")
mypy

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
device

## Tokenization

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

    Tokenizing the whole dataset

In [ ]:
#Tokenize the dataset to the correct input for the transformer model.
def tokenize(batch):
    return tokenizer(batch["text_clean"], padding="max_length", truncation=True)

In [ ]:
rust_tokenized = rust.map(tokenize, batched=True)
rust_testset = rust_tokenized["test"]
rust_testset

In [ ]:
flutter_tokenized = flutter.map(tokenize, batched=True)
flutter_testset = flutter_tokenized["test"]
flutter_testset

In [ ]:
mypy_tokenized = mypy.map(tokenize, batched=True)
mypy_testset = mypy_tokenized["test"]
mypy_testset

In [ ]:
tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset

In [ ]:
# List the columns of the dataset.
# Should be: ["attention_mask", "input_ids", "labels", "token_type_ids"]
# Remove unnecessary columns that the model does not expect.
tokenized_dataset = tokenized_dataset.remove_columns(["text_clean", "__index_level_0__"])

train_dataset = tokenized_dataset["train"]
print(train_dataset)
validation_dataset = tokenized_dataset["validate"]
print(validation_dataset)
test_dataset = tokenized_dataset["test"]
test_dataset

## Training a classifier

In [ ]:
training_args = TrainingArguments(
    output_dir=model_dir,
    num_train_epochs=epochs,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    save_strategy=save_strategy,
    save_steps=save_steps,
    evaluation_strategy="epoch",
    learning_rate=learning_rate,
    logging_steps=logging_steps,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
)

In [ ]:
trainer.train() 

* Training loss: Difference between the predictons made by the model on the training dataset vs on the actual data.
* Validation loss: how well the model functions on unseen data.
* Accuracy: How much the model gets correct. number of correct Prediction / total number of predictions.
* F1: consider both precision and recall. 
* Precision: Accuracy of positive predictions. Percison TP = TP + FP. How often the model is correct.
* Recall: True positive rate. how many items the model gets correct from the total amount.

### Training loss decreases, validation loss increases = Overfitting

In [ ]:
# Evaluate validation set
eval_result = trainer.evaluate(eval_dataset=validation_dataset)

In [ ]:
for key, value in sorted(eval_result.items()):
    print(f"{key} = {value}\n")

In [ ]:
# Evaluate test data set
test_results = trainer.evaluate(eval_dataset=test_dataset)

In [ ]:
for key, value in sorted(test_results.items()):
    print(f"{key} = {value}\n")

In [ ]:
rust_test_results = trainer.evaluate(eval_dataset=rust_testset)
for key, value in sorted(rust_test_results.items()):
    print(f"{key} = {value}\n")    

In [ ]:
flutter_test_results = trainer.evaluate(eval_dataset=flutter_testset)
for key, value in sorted(flutter_test_results.items()):
    print(f"{key} = {value}\n")

In [ ]:
mypy_test_results = trainer.evaluate(eval_dataset=mypy_testset)
for key, value in sorted(mypy_test_results.items()):
    print(f"{key} = {value}\n")

In [ ]:
trainer.save_model(model_dir + "_local") 

In [ ]:
from transformers import pipeline
    
classifier = pipeline("text-classification", model="./model_local")

In [ ]:
classifier.model

In [ ]:
classifier("this does not need to be done fast")

In [ ]:
classifier("this is super important")

In [ ]:
classifier("this bug has super high impact on the project")

## Important to delete large objects to free memory 
del train_dataset

In [ ]:
del validation_dataset 

In [ ]:
del model

In [ ]:
# Free cache
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi